Importing the required Libraries

In [55]:
import pandas as pd
import numpy as np
import random
import torch, torchtext
from torchtext import data 

Uploading the Stanford dataset and extracting the labels

In [56]:
dataset_sentences = pd.read_csv('/content/datasetSentences.txt', sep='\t')
print(dataset_sentences.shape, '\n', dataset_sentences.head())

sentiment_labels = pd.read_csv('/content/sentiment_labels.txt', sep='|')                
sentiment_labels = sentiment_labels.rename(columns={'phrase ids': 'phrase_id', 
                                                    'sentiment values': 'sentiment_value'})
print(sentiment_labels.shape, '\n', sentiment_labels.head())

(11855, 2) 
    sentence_index                                           sentence
0               1  The Rock is destined to be the 21st Century 's...
1               2  The gorgeously elaborate continuation of `` Th...
2               3                     Effective but too-tepid biopic
3               4  If you sometimes like to go to the movies to h...
4               5  Emerges as something rare , an issue movie tha...
(239232, 2) 
    phrase_id  sentiment_value
0          0          0.50000
1          1          0.50000
2          2          0.44444
3          3          0.50000
4          4          0.42708


Preparing the dataset

In [57]:
sentence_sentiment = dataset_sentences
phrase_sentiment = dict(zip(list(sentiment_labels.phrase_id), 
                            list(sentiment_labels.sentiment_value)))


with open('/content/STree.txt') as file:
    stree = file.readlines()
stree = [line.strip().split('|') for line in stree] 

Phrase to Sentence mapping

In [58]:
from statistics import mean

# average sentiments per sentence (otherwise long sentences get high sentiments)
sentiments = [mean(phrase_sentiment[int(phrase_id)] for phrase_id in phrase_ids) 
              for phrase_ids in stree]
# define neutral sentiment as values within [0.5, 0.55) 
sentiments = [2 if sentiment >= 0.5 and sentiment < 0.55 else sentiment 
              for sentiment in sentiments]
# all negative sentiment values to 0, all positive sentiment values to 1
sentiments = [int(round(sentiment)) 
              for sentiment in sentiments]
# concatenate sentence sentiment values with corresponding sentence texts
sentence_sentiment['sentiment_value'] = sentiments

sentence_sentiment

,sentence_index,sentence,sentiment_value
0,1,The Rock is destined to be the 21st Century 's...,0
1,2,The gorgeously elaborate continuation of `` Th...,0
2,3,Effective but too-tepid biopic,0
3,4,If you sometimes like to go to the movies to h...,0
4,5,"Emerges as something rare , an issue movie tha...",0
...,...,...,...
11850,11851,A real snooze .,0
11851,11852,No surprises .,0
11852,11853,We 've seen the hippie-turned-yuppie plot befo...,0
11853,11854,Her fans walked out muttering words like `` ho...,0


In [59]:
sentence_sentiment = sentence_sentiment.rename(columns={'sentence': 'texts', 'sentiment_value': 'labels'})
sentence_sentiment = sentence_sentiment[['texts', 'labels']]


In [60]:
sentence_sentiment

,texts,labels
0,The Rock is destined to be the 21st Century 's...,0
1,The gorgeously elaborate continuation of `` Th...,0
2,Effective but too-tepid biopic,0
3,If you sometimes like to go to the movies to h...,0
4,"Emerges as something rare , an issue movie tha...",0
...,...,...
11850,A real snooze .,0
11851,No surprises .,0
11852,We 've seen the hippie-turned-yuppie plot befo...,0
11853,Her fans walked out muttering words like `` ho...,0


In [61]:
SEED = 1234
torch.manual_seed(SEED)

Defining Text & Label fields

In [62]:
Text = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [63]:
fields = [('texts', Text),('labels',Label)]

In [64]:
#example = [torchtext.legacy.data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])]
example = [torchtext.legacy.data.Example.fromlist([sentence_sentiment.texts[i],sentence_sentiment.labels[i]], fields) for i in range(sentence_sentiment.shape[0])]


In [65]:
stanfordDataset = torchtext.legacy.data.Dataset(example, fields)

In [66]:
(train, test) = stanfordDataset.split(split_ratio=[70, 30], random_state = random.seed(SEED))

Data split for train & test

In [67]:
(len(train), len(test))

(8298, 3557)

In [68]:
vars(train.examples[10])

{'labels': 0,
 'texts': ['Not',
  'only',
  'are',
  'the',
  'film',
  "'s",
  'Sopranos',
  'gags',
  'incredibly',
  'dated',
  'and',
  'unfunny',
  ',',
  'they',
  'also',
  'demonstrate',
  'how',
  'desperate',
  'the',
  'makers',
  'of',
  'this',
  '`',
  'we',
  "'re",
  '-',
  'doing',
  '-',
  'it',
  '-',
  'for',
  '-',
  'the',
  '-',
  'cash',
  "'",
  'sequel',
  'were',
  '.']}

Vocabulary build-up

In [69]:
Text.build_vocab(train)
Label.build_vocab(train)

In [70]:
import random

In [71]:
MAX_VOCAB_SIZE = 12_000

Text.build_vocab(train, max_size = MAX_VOCAB_SIZE)
Label.build_vocab(train)

In [72]:
print('Size of input vocab : ', len(Text.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Text.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  12002
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 7825), (',', 6898), ('the', 6012), ('and', 4411), ('of', 4380), ('a', 4214), ('to', 3009), ('-', 2740), ("'s", 2474), ('is', 2453)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [74]:
train_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits((train, test), batch_size = 16, 
                                                            sort_key = lambda x: len(x.texts),
                                                            sort_within_batch=True, device = device)

In [75]:
next(iter(train_iterator))


[torchtext.legacy.data.batch.Batch of size 16]
	[.texts]:('[torch.LongTensor of size 16x10]', '[torch.LongTensor of size 16]')
	[.labels]:[torch.LongTensor of size 16]

In [76]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Text.vocab.stoi, tokens)

Model

In [77]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                               hidden_dim, 
                               num_layers=n_layers, 
                               dropout=dropout,
                               batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function (softmax)
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

Hyper-Parameters

In [78]:
# Define hyperparameters
size_of_vocab = len(Text.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

print(model)

classifier(
  (embedding): Embedding(12002, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)


In [79]:
#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,842,503 trainable parameters


In [80]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

Train the Model

In [81]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.texts   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluate the Model

In [82]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.texts
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Run the Model

In [83]:
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    # valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # # save the best model
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 0.992 | Train Acc: 56.31%
	Train Loss: 0.925 | Train Acc: 61.69%
	Train Loss: 0.796 | Train Acc: 75.53%
	Train Loss: 0.768 | Train Acc: 77.78%
	Train Loss: 0.738 | Train Acc: 81.54%
	Train Loss: 0.716 | Train Acc: 84.15%
	Train Loss: 0.704 | Train Acc: 84.72%
	Train Loss: 0.699 | Train Acc: 84.86%
	Train Loss: 0.695 | Train Acc: 85.01%
	Train Loss: 0.663 | Train Acc: 89.68%
	Train Loss: 0.618 | Train Acc: 94.46%
	Train Loss: 0.595 | Train Acc: 96.50%
	Train Loss: 0.583 | Train Acc: 97.94%
	Train Loss: 0.577 | Train Acc: 98.66%
	Train Loss: 0.572 | Train Acc: 98.89%
	Train Loss: 0.573 | Train Acc: 98.65%
	Train Loss: 0.567 | Train Acc: 99.01%
	Train Loss: 0.569 | Train Acc: 98.76%
	Train Loss: 0.564 | Train Acc: 99.16%
	Train Loss: 0.563 | Train Acc: 99.11%


In [84]:
torch.save(model.state_dict(), 'saved_weights.pt')

In [85]:
model.load_state_dict(torch.load('saved_weights.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.680 | Test Acc: 86.79%


In [86]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(texts):
    
    categories = {0: "Very Negative", 1:"Negative", 2:"Neutral", 3:"Positive", 4:"Very Positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(texts)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

User Input Testing

In [87]:
classify_sentence("A valid explanation for why Trump won't let women on the golf course.")

'Neutral'

In [88]:
classify_sentence("In his teen years, Obama has been known to use marijuana and cocaine.")

'Negative'

In [89]:
classify_sentence("A masterpiece four years in the making.")

'Negative'

In [90]:
classify_sentence("The movie 's ripe , enrapturing beauty will tempt those willing to probe its inscrutable mysteries.")

'Negative'